<a href="https://colab.research.google.com/github/Md819-max/site/blob/main/Bitcoin_june_24_to_next_5days_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd

# Download BTC-USD data for the last 5 years
btc_data = yf.download("BTC-USD", period="5y", interval="1d")

# Save to CSV
btc_data.to_csv("btc_timeseries_5y.csv")

# Show the first few rows
print(btc_data.head())


/tmp/ipython-input-27-1507147689.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  btc_data = yf.download("BTC-USD", period="5y", interval="1d")
[*********************100%***********************]  1 of 1 completed

Price             Close         High          Low         Open       Volume
Ticker          BTC-USD      BTC-USD      BTC-USD      BTC-USD      BTC-USD
Date                                                                       
2020-06-24  9313.610352  9680.367188  9278.233398  9632.149414  18961716076
2020-06-25  9264.813477  9340.161133  9095.324219  9314.126953  18616048626
2020-06-26  9162.917969  9310.516602  9101.738281  9260.995117  18341465837
2020-06-27  9045.390625  9207.810547  8998.216797  9167.824219  17273093144
2020-06-28  9143.582031  9197.546875  8975.525391  9048.460938  14560870760


In [ ]:
pip install yfinance


In [ ]:
df = pd.read_csv('/content/btc_timeseries_5y.csv')

In [ ]:
df.head(2)

,Price,Close,High,Low,Open,Volume
0,Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
1,Date,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.columns

Index(['Price', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, silhouette_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt

# Download Bitcoin data (5 years)
data = yf.download("BTC-USD", period="5y", interval="1d").dropna()

# Create features
data["Return"] = data["Close"].pct_change()
data["Target"] = np.where(data["Close"].shift(-5) > data["Close"], 1, 0)
data.dropna(inplace=True)

# Features and labels
X = data[["Close", "Return"]]
y = data["Target"]

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Logistic Regression
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_pred))

# Decision Tree
dt = DecisionTreeClassifier(max_depth=5)
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)
print("Decision Tree Accuracy:", accuracy_score(y_test, dt_pred))

# Random Forest
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))

# KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
print("KNN Accuracy:", accuracy_score(y_test, knn_pred))

# KMeans Clustering (Unsupervised)
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_scaled)
print("KMeans Silhouette Score:", silhouette_score(X_scaled, kmeans.labels_))

# Deep Learning - Classification
model_cls = Sequential()
model_cls.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model_cls.add(Dense(32, activation='relu'))
model_cls.add(Dense(1, activation='sigmoid'))
model_cls.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_cls.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
cls_loss, cls_acc = model_cls.evaluate(X_test, y_test, verbose=0)
print("Deep Learning Classification Accuracy:", cls_acc)

# Deep Learning - Regression (Predict future Close)
future_target = data["Close"].shift(-5).dropna()
data_reg = data.iloc[:-5]
X_reg = scaler.fit_transform(data_reg[["Close", "Return"]])
y_reg = future_target.values
# Corrected typo: train_test_spl to train_test_split
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

/tmp/ipython-input-35-1220162971.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("BTC-USD", period="5y", interval="1d").dropna()
[*********************100%***********************]  1 of 1 completed


Logistic Regression Accuracy: 0.5300546448087432
Decision Tree Accuracy: 0.5737704918032787
Random Forest Accuracy: 0.5491803278688525
KNN Accuracy: 0.5655737704918032
KMeans Silhouette Score: 0.38885983258148465


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Deep Learning Classification Accuracy: 0.5136612057685852


In [ ]:
from tensorflow.keras.layers import LSTM, Dropout

model_lstm = Sequential()
model_lstm.add(LSTM(50, return_sequences=True, input_shape=(X_train_r.shape[1], 1)))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(50))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer='adam', loss='mse')
model_lstm.fit(X_train_r.reshape(-1, X_train_r.shape[1], 1), y_train_r, epochs=100, batch_size=32)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 2653163776.0000
Epoch 2/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2587018496.0000
Epoch 3/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2666093056.0000
Epoch 4/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2720317440.0000
Epoch 5/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2667274752.0000
Epoch 6/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2726585088.0000
Epoch 7/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2522400512.0000
Epoch 8/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2604591872.0000
Epoch 9/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2685271552.0000
Epoch 10/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2650372608.0000
Epoch 11/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2774384896.0000
Epoch 12/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2737942272.0000
Epoch 13/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2727914240.0000
Epoch 14/100
46/46 ━━

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt

# Download Bitcoin data
data = yf.download("BTC-USD", period="5y", interval="1d").dropna()

# Feature Engineering — Add Technical Indicators
data["Return"] = data["Close"].pct_change()
data["MA7"] = data["Close"].rolling(window=7).mean()
data["MA21"] = data["Close"].rolling(window=21).mean()
data["EMA"] = data["Close"].ewm(span=20, adjust=False).mean()
data["Volatility"] = data["Close"].rolling(window=7).std()
data["Target"] = np.where(data["Close"].shift(-5) > data["Close"], 1, 0)
data.dropna(inplace=True)

# Features and labels
X = data[["Close", "Return", "MA7", "MA21", "EMA", "Volatility"]]
y = data["Target"]

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Hyperparameter Tuning for Random Forest
from sklearn.ensemble import RandomForestClassifier
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5]
}
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best Model
best_rf = grid_search.best_estimator_
rf_pred = best_rf.predict(X_test)
print("Tuned Random Forest Accuracy:", accuracy_score(y_test, rf_pred))
print("Best RF Params:", grid_search.best_params_)

# Deep Learning — LSTM for Regression (Predict Close price)
# Prepare Data
future_target = data["Close"].shift(-5).dropna()
data_reg = data.iloc[:-5]
X_reg = scaler.fit_transform(data_reg[["Close", "Return", "MA7", "MA21", "EMA", "Volatility"]])
y_reg = future_target.values

# Reshape for LSTM: (samples, timesteps, features)
X_reg = X_reg.reshape(X_reg.shape[0], 1, X_reg.shape[1])

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Build LSTM Model
model_lstm = Sequential()
model_lstm.add(LSTM(64, return_sequences=True, input_shape=(X_train_r.shape[1], X_train_r.shape[2])))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(32))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(1))

model_lstm.compile(optimizer='adam', loss='mse')

# Train
model_lstm.fit(X_train_r, y_train_r, epochs=100, batch_size=32, verbose=0)

# Predict
pred_lstm = model_lstm.predict(X_test_r)
rmse = np.sqrt(mean_squared_error(y_test_r, pred_lstm))
print("LSTM Regression RMSE:", rmse)


/tmp/ipython-input-37-913592420.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("BTC-USD", period="5y", interval="1d").dropna()
[*********************100%***********************]  1 of 1 completed


Tuned Random Forest Accuracy: 0.6823204419889503
Best RF Params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
LSTM Regression RMSE: 53930.50947900477


In [ ]:
# Get the last available input data
last_input = X_reg[-1].reshape(1, 1, X_reg.shape[2])

# Empty list for predictions
future_prices = []

for _ in range(5):
    pred_price = model_lstm.predict(last_input)[0][0]
    future_prices.append(pred_price)

    # Update last_input by shifting and inserting the predicted value
    new_close = (pred_price - scaler.mean_[0]) / scaler.scale_[0]  # inverse scaling for Close
    new_features = np.array([[new_close, 0, 0, 0, 0, 0]])  # you can also estimate or carry last known values for other features
    new_features_scaled = new_features.reshape(1, 1, X_reg.shape[2])

    last_input = new_features_scaled

# Show predicted 5 days' BTC Close prices
print("Predicted Next 5 Days' Close Prices (approx):")
for i, price in enumerate(future_prices, 1):
    print(f"Day {i}: {price:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Predicted Next 5 Days' Close Prices (approx):
Day 1: 127.48
Day 2: 127.48
Day 3: 127.48
Day 4: 127.48
Day 5: 127.48


In [ ]:
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# Download Bitcoin Close price
data = yf.download("BTC-USD", period="5y", interval="1d")["Close"].values.reshape(-1,1)

# Scale data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Prepare sequences
X, y = [], []
for i in range(60, len(data_scaled)-5):
    X.append(data_scaled[i-60:i, 0])
    y.append(data_scaled[i, 0])
X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split last 60 values for forecasting
forecast_input = data_scaled[-60:].reshape(1, 60, 1)

# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=50, batch_size=32, verbose=0)

# Predict next 5 days recursively
future_prices = []
input_seq = forecast_input.copy()

for _ in range(5):
    pred_price = model.predict(input_seq)[0][0]
    future_prices.append(pred_price)
    input_seq = np.append(input_seq[:,1:,:], [[[pred_price]]], axis=1)

# Inverse scale predictions
future_prices_actual = scaler.inverse_transform(np.array(future_prices).reshape(-1,1))

# Show predictions
print("Predicted Next 5 Days Close Prices:")
for i, price in enumerate(future_prices_actual.flatten(), 1):
    print(f"Day {i}: ${price:.2f}")


/tmp/ipython-input-39-3794307110.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("BTC-USD", period="5y", interval="1d")["Close"].values.reshape(-1,1)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Predicted Next 5 Days Close Prices:
Day 1: $103591.48
Day 2: $102730.22
Day 3: $101272.60
Day 4: $99687.72
Day 5: $98133.59


In [ ]:
!pip install plotly


ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Your predicted Close prices
predicted_close = [103591.48, 102730.22, 101272.60, 99687.72, 98133.59]

# Generate synthetic Open, High, Low around Close for visualization
np.random.seed(42)
predicted_open = [c + np.random.uniform(-500, 500) for c in predicted_close]
predicted_high = [max(o, c) + np.random.uniform(200, 800) for o, c in zip(predicted_open, predicted_close)]
predicted_low  = [min(o, c) - np.random.uniform(200, 800) for o, c in zip(predicted_open, predicted_close)]

# Create a dataframe
dates = pd.date_range(start=pd.Timestamp.today(), periods=5)
df = pd.DataFrame({
    'Date': dates,
    'Open': predicted_open,
    'High': predicted_high,
    'Low': predicted_low,
    'Close': predicted_close
})

# Create Candlestick chart
fig = go.Figure(data=[go.Candlestick(
                x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.update_layout(title='Next 5 Days Bitcoin Price Prediction',
                  xaxis_title='Date',
                  yaxis_title='Price (USD)',
                  xaxis_rangeslider_visible=False)

fig.show()
